In [289]:
import utils

warnings.filterwarnings(action="ignore")
os.getcwd()

'/Users/yangwoolee/git_repo/seoul_bike_dash/moon_light'

> ### 자료 불러오기

In [290]:
station = pd.read_csv('assets/seoul_bike_station_01_12.csv',encoding="CP949",index_col=0)
near_bus = pd.read_csv('assets/near_bus_500m.csv',encoding="CP949",index_col=0)
seoul_bike = pd.read_parquet('/Users/yangwoolee/git_repo/seoul_bike_dash/parquet/220607_bike_record.parquet.gzip')

> ### 이름으로 버스 정류장과 대여소 선택

In [267]:
# 출발, 도착
starting_point = near_bus[near_bus['name'].str.contains('합정역')] # 버스, 대여소, 역 모두 넣어놓는 리스트로 만들기
ending_bike = 754
ending_point = station.query("st_id==@ending_bike")


def finding_start_end(starting_point,ending_point) :
    '''
    최종적으로 자전거 대여소끼리 이동하는 경로를 찾는 함수
    목적지 대여소와 가까운 버스정류소를 찾고 그 근처의 대여소를 추천해준다.
    
    bus에서 내린 뒤 대여소로 이동하는 경우를 산정함

    starting_point : 출발지 검색용 변수 
    ending_bike : 목적지 대여소 번호
    ending_point = 목적지 대여소 정보 
    '''

    # 목적지 대여소와 직선거리 1.5km 이내 버스대여소 모두 선택
    # 결과 : bus_route_near_bike
    result_km = haversine_np(
        ending_point["longtitude"].values,
        ending_point["latitude"].values,
        near_bus["longtitude"].values,
        near_bus["latitude"].values,
    )
    near_bus_for_dist = near_bus.copy()
    near_bus_for_dist["dist"] = result_km
    bus_route_near_bike = (
        near_bus_for_dist[near_bus_for_dist["dist"] < 1500]
        .dropna()
        .sort_values(by="id")
    )

    # starting_point와 ending_point 인근 버스 정류장과 겹치는 버스 번호 찾기
    # ex) 합정역 => 754 자전거 대여소 인 경우 754 근처를 지나가는 버스 N26과 N62를 결과로 제공함.
    bus_route_inter = np.intersect1d(
        bus_route_near_bike["bus"].unique(), starting_point["bus"].unique()
    ).tolist()

    # 주변에 겹치는 결과가 없으면 검색 종료(검색 반경을 늘려서 검색하도록 유도할 수도 있겠다. 기능 추가할때 구현고민해보자.)
    if bus_route_inter == [] : 
        print('검색결과 없음')

        # 정류장,지하철역, 대여소 등 관련 내용이 없으면 station => station 검색 함수로 옮김
        return print('station_to_station')

    # 겹치는 노선에 대한 정보만 불러온더 
    # 합정의 경우 N26,N62외에도 다양한 야간 버스가 지나친다. 그중 N26,N62버스 만 불러온다.
    starting_bus = starting_point.query("bus == @bus_route_inter")
    ending_bus = bus_route_near_bike.query("bus == @bus_route_inter")
    

    # 정류장 시작,종료 지점 모으기.. 머리 간질간질해 죽는줄..
    # A에서 B갈때 거치는 정류장과 B에서 A갈때 거치는 정류장이 다를 수 있음.
    # 정확한 버스노선을 찾기 위해서는 버스 방향이 상행인지 하행인지가 중요하다. 
    # 상행과 하행을 찾는 방법에 대한 방법이다. 

    bus_df = pd.DataFrame(columns=ending_bus.columns)
    # bus_route_inter : A->B갈때 여러 N버스를 타고 갈 수 있음
    # 합정 => 754인 경우 N26과 N62 둘 다 가능 
    for num in bus_route_inter:
        starting_order = starting_bus.query("bus==@num")
        ending_order = ending_bus.sort_values(by=["dist",'order']).query("bus==@num")[:2]

        # 상행(A->B)인 경우 B가 A보다 bus 정류장 번호(column : bus_order)가 더 크다.
        # 하행(B->A)인 경우 A가 B보다 "" 

        # 그 이유는 버스가 이동할때마다 +1씩 증가하기 때문임. 운행중에는 번호가 계속 증가하는 순으로 갔다가 
        # 차고지에서 다시 0으로 리셋하고 출발한다는 생각을 하면 쉬움. 

        # 따라서 하행인 경우 ending_order .max - starting.max > 0 이며 상행인 경우 그 반대임.

        #하행노선 찾기
        if ending_order['order'].max() - starting_order['order'].max() >= 0 :
            df = starting_order.sort_values(by='order',ascending=False)[:1]
            bus_df = bus_df.append(df)
            df = ending_order.sort_values(by='order',ascending=False)[:1]
            bus_df = bus_df.append(df)
            print( f'{num}하행 노선입니다.')

        #상행노선 찾기
        elif ending_order['order'].max() - starting_order['order'].max() < 0 :
            df = starting_order.sort_values(by='order')[:1]
            bus_df = bus_df.append(df)
            df = ending_order.sort_values(by='order')[:1]
            bus_df = bus_df.append(df)
            print('상행 노선입니다.')

        
    # 출발지 위치가 목적지 1.5km이내에 있는 경우 버스 경로검색 안하도록 조치
    if starting_order['id'].iloc[0] in bus_route_near_bike['id'].tolist() :
        bus_df = starting_order.sort_values(by='order',ascending=False)[:1]
        print('목적지와 정류장이 직선거리 1.5km 이내에 있습니다.')
        # 아래 함수랑 연동시켜보자
        return bus_df        

    # 아래 함수랑 연동시켜보자
    return bus_df #  노선별 출발 => 도착를 return 한다.


pick_destination=finding_start_end(starting_point, ending_point)

# 추천 dist 가까운 순으로 하기 
# 만약 112클릭하면 111,112,113 버스정류장과 근처 따릉이 지도에 표시하기 
# 112와 그 주변 따릉이 색은 다른곳과 다르게 찐하게 표시하기


pick_destination

N26하행 노선입니다.
N62하행 노선입니다.


,bike_id,bus,order,id,name,longtitude,latitude,dist
523,"[104,105,106,497,4211]",N26,108,14012,합정역,126.913997,37.549682,NaN
527,"[700,773,1169,1171,2703,3761]",N26,112,16001,염창역.서울도시가스,126.875881,37.547097,665.465568
1388,"[104,105,106,497,4211]",N62,135,14012,합정역,126.913997,37.549682,NaN
1391,"[700,773,1169,1171,2703,3761]",N62,138,16001,염창역.서울도시가스,126.875881,37.547097,665.465568


In [268]:
# 원하는 버스노선 선택(list item에서 선택한다는 생각)
pick = pick_destination[:2]

In [269]:
# 지도에 경로 표시하기 위한 데이터 추출용 함수임.
# 결과 
# route_whole : 전체 이동경로
# route_middle : 중간 지점(위치 정확도를 위해)
# route_start_and_end : 경로 출발, 종료지점 

def bus_route_info(pick): 
    if pick.iloc[-1]["id"] == pick.iloc[0]["id"]:
        print("버스를 타지 않았네")
        return [],[],[]
    else:
        # 범위선택
        bus_order_start = pick["order"].iloc[0]
        bus_order_end = pick["order"].iloc[-1]
        bus_name = pick["bus"].iloc[0]

        # 자료 sorting
        route_whole = near_bus.query(
            "@bus_order_start <= order <= @bus_order_end & bus == @bus_name"
        )[["name", "longtitude", "latitude",'order']].sort_values(by='order')

        route_start = route_whole.iloc[0]
        route_end = route_whole.iloc[-1]
        route_start_and_end = pd.concat([route_start,route_end],axis=1).T

        route_middle = route_whole.iloc[int(len(route_whole) / 2)]

        #return
        bus_route = [route_whole, route_middle,route_start_and_end]

    return bus_route

route_whole, route_middle,route_start_and_end = bus_route_info(pick)
# route end name은 출발, 끝 지점 텍스트 표시할때 사용하면 될듯

In [270]:
## 외부 함수
# 위에있는 데이터프레임이 들어가야함


def bike_to_bike(bike_id , bike_station_id) :
    ''' 
    추천 대여소 정보를 만드는 함수임

    bike_id : 가능한 자전거 대여소 id
    bike_station_id : 목적지 대여소 id 
    '''
    
    # 추천_대여소_선정 : ast : '[123,]' => [123]으로 만들어줌 --> list
    if type(bike_id) == str :
        bike_id = ast.literal_eval(bike_id)
    
    # 버스정류장 근처 대여소와 집 근처 대여소 거리 비교
    # ending point를 외부에서 받아야하네 (클래스로 함수를 만들어야하나.)

    # 추천 대여소 리스트 만들기 
    BM_2 = (seoul_bike['st_id2'] == bike_station_id) & (seoul_bike['st_id1'].isin(bike_id))
    BM_1 = (seoul_bike['st_id1'] == bike_station_id) & (seoul_bike['st_id2'].isin(bike_id))
    seoul_bike_sorting = seoul_bike.loc[BM_1 | BM_2] # 거리 및 시간 계산을 위해 따로 추출


    bike_recommend =[]
    for starting_station in bike_id:
        BM_1 = (seoul_bike_sorting["st_id1"] == starting_station) & (seoul_bike_sorting["st_id2"] == bike_station_id)
        BM_2 = (seoul_bike_sorting["st_id2"] == starting_station) & (seoul_bike_sorting["st_id1"] == bike_station_id)

        # 평균시간 계산(column : time)
        mean_station = (
            seoul_bike_sorting[BM_1 | BM_2]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)[:5]
            .index.values.mean()
        )

        # 이용기록 계산(column : num)
        num_station = len(seoul_bike_sorting[BM_1 | BM_2])

        # 직선거리 계산(column : dist)
        lat_1 = station.query("st_id==@starting_station")["latitude"].values
        lon_1 = station.query("st_id==@starting_station")["longtitude"].values
        lat_2 = station.query("st_id==@bike_station_id")["latitude"].values
        lon_2 = station.query("st_id==@bike_station_id")["longtitude"].values
        dist_station = haversine_np(lat_1, lon_1, lat_2, lon_2)
        
        #자료 저장
        station_info = station.query('st_id==@starting_station')
        bike_recommend.append([station_info['st_name'].iloc[0],station_info['latitude'].iloc[0],station_info['longtitude'].iloc[0],starting_station, mean_station,num_station,f"{dist_station[0]/1000 :.2f}km"])

    #return
    bike_recommend = pd.DataFrame(bike_recommend,columns=['st_name','latitude','longtitude','st_id','time','num','dist'])

    return bike_recommend



# 16001 염창
# 13045 충정로
# starting_bus_id = 16002
bike_id = pick.iloc[-1]['bike_id'] # bike_id는 무조건 리스트 형태로
ending_bike_id = 754
bike_recommend = bike_to_bike(bike_id, ending_bike_id).sort_values(by='dist')




In [271]:
# # trick figure
# fig = px.scatter_mapbox(station.query("st_id == 207"), lat="latitude", lon="longtitude",zoom=12)
# fig.update_traces(
#     marker=go.scattermapbox.Marker(symbol="circle", size=10, opacity=0.001, color="white")
# )

# fig.update_layout(
#     mapbox=dict(
#         accesstoken="pk.eyJ1IjoieWFuZ29vcyIsImEiOiJjbDNqd2tkN2IwbGdmM2pvNzF0c2M4NnZkIn0.J3IjPYg3w28cGiWkUD7bnA",
#         # style='mapbox://styles/yangoos/cl3jubvl7000c14llgtoev0nm'
#     ),
#     margin=dict(l=20, r=20, t=20, b=20)
# )
# fig.show()

In [279]:
arrival.iloc[0]['latitude']

37.541142

In [282]:
import requests


def bike_route(departure, arrival) :
    url = "https://apis.openapi.sk.com/tmap/routes/pedestrian?version=1"

    payload = {
        "angle": 0,
        "speed": 0,
        "reqCoordType": "WGS84GEO",
        "searchOption": "0",
        "resCoordType": "WGS84GEO",
        "sort": "index",
        "startX": departure.iloc[0]['longtitude'],
        "startY": departure.iloc[0]['latitude'],
        "endX": arrival.iloc[0]['longtitude'],
        "endY": arrival.iloc[0]['latitude'],
        "startName": '출발',
        "endName": '도착'
    }
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "appKey": "l7xxfdc75c1509a74ecdba02bf5e024ee9d5"
    }

    response = requests.post(url, json=payload, headers=headers).text

    api_data = json.loads(response).get('features')

    coor_raw_data = [api_data[i].get('geometry').get('coordinates') for i in range(0,len(api_data))]

    finish_data = []
    for data in coor_raw_data :
        # data = api_data[1].get('geometry').get('coordinates')
        if type(data[0]) == list :
            for i in data:
                finish_data.append(i)
        else :
            pass
    return pd.DataFrame(finish_data).T.values

In [285]:
# trick figure
fig = px.scatter_mapbox(station.query("st_id == 207"), lat="latitude", lon="longtitude")
fig.update_traces(
    marker=go.scattermapbox.Marker(symbol="circle", size=10, opacity=0.001, color="white")
)


# -- 추천 대여소 그리기 -- #
# recomend_id = result_recommend['id'].tolist()
# fig = px.scatter_mapbox(station.query("id == @recomend_id"), lat="latitude", lon="longtitude")
# fig.update_traces(
#     marker=go.scattermapbox.Marker(symbol="circle", size=10, opacity=0.7, color="blue")
# )


# -- 대중교통 추천 경로 그리기 -- # 
def get_optimal_route(start, goal, waypoints=["", ""], option="traoptimal"):
    client_id = '44vlsxxinm'
    client_secret = 'BOHC6psFnLkjdRrlkLn24S1apLUTsuzrCnMuCi4A'
    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?start={start[0]},{start[1]}&waypoint={waypoints[0]},{waypoints[1]}&goal={goal[0]},{goal[1]}&option={option}"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    request.add_header("X-NCP-APIGW-API-KEY", client_secret)

    response = urllib.request.urlopen(request)
    res = response.getcode()

    if res == 200:
        response_body = response.read().decode("utf-8")
        return json.loads(response_body)

    else:
        print("ERROR")


#변수 
# option : 탐색옵션 [최대 3개, traoptimal(기본 옵션)
# / trafast, tracomfort, traavoidtoll, traavoidcaronly]
option = "traoptimal"
starting_bus = route_start_and_end[['longtitude','latitude']].values[0]
ending_bus = route_start_and_end[['longtitude','latitude']].values[1]
way_bus  = route_middle[['longtitude','latitude']].values

result = get_optimal_route(starting_bus, ending_bus,way_bus)


a = result.get('route').get(option)[0].get('path')
b = pd.DataFrame(a)

fig_1 = go.Figure(
    go.Scattermapbox(
        lat=b[1],
        lon=b[0],
        mode="lines",
        marker=dict(symbol="circle", size=15, color='#AD00FF' , opacity=0.5),
    )
)
fig.add_trace(fig_1.data[0])


# -- bus 정류장 그리기 --#
fig_2 = px.scatter_mapbox(route_start_and_end, lat="latitude", lon="longtitude", hover_name="name",zoom=11,height=500
)
fig_2.update_traces(
    marker=go.scattermapbox.Marker(symbol="circle", size=10, opacity=0.7, color="#AD00FF")
)
fig.add_trace(fig_2.data[0])


# --반경 내 따릉이 대여소 그리기--#

# 거리 계산
station_haversine = haversine_np(
    pick.iloc[-1]["longtitude"],
    pick.iloc[-1]["latitude"],
    station["longtitude"].values,
    station["latitude"].values,
)

# 데이터 추가
around_station = station.copy()
around_station['dist'] = station_haversine
around_station = around_station[around_station['dist'] < 500] 

# 시각화
fig_3 = go.Figure(
    go.Scattermapbox(
        name=f"station with in {round(500/1000,2)}km",
        lat=around_station["latitude"],
        lon=around_station["longtitude"],
        mode="markers",
        # marker = {'size': 20, 'symbol': "bus"},
        marker=go.scattermapbox.Marker(size=8, color="black", opacity=0.2),
        text=around_station["st_name"],
    )
)
fig.add_trace(fig_3.data[0])

# --목적지 대여소 그리기 -- #
fig_4 = go.Figure(
    go.Scattermapbox(
        name="location",
        lat=ending_point["latitude"],
        lon=ending_point["longtitude"],
        mode="markers",
        # marker = {'size': 20, 'symbol': "bus"},
        marker=go.scattermapbox.Marker(size=10, color="fuchsia", opacity=0.8),
        text=ending_point["st_name"].values,
    )
)


departure = bike_recommend
arrival = ending_point

data_bike_route = bike_route(departure=departure, arrival=arrival)
fig_5 = go.Figure(
    go.Scattermapbox(
        lat=data_bike_route[1],
        lon=data_bike_route[0],
        mode="lines",
        marker=dict(symbol="circle", size=15, color='black' , opacity=0.5),
    )
)
fig.add_trace(fig_5.data[0])



fig.add_trace(fig_4.data[0])

fig.update_layout(
    mapbox=dict(
        accesstoken="pk.eyJ1IjoieWFuZ29vcyIsImEiOiJjbDNqd2tkN2IwbGdmM2pvNzF0c2M4NnZkIn0.J3IjPYg3w28cGiWkUD7bnA",
        # style='mapbox://styles/yangoos/cl3jubvl7000c14llgtoev0nm'
    ),
    margin=dict(l=20, r=20, t=20, b=20)
)
fig.show()


In [ ]:
ending_point  = station.query("id == @ending_bike_id")[['경도','위도']].values[0]

starting_point = result_recommend[['경도','위도']].values[0]


126.874489

> ### 길찾기 기능(비추..)

In [244]:
a = [["합정역", 126.913997,  37.549682] 
,["당산역", 126.902410, 37.534863]]

In [255]:
import requests


def bike_route(departure, arrival) :
    url = "https://apis.openapi.sk.com/tmap/routes/pedestrian?version=1"

    payload = {
        "angle": 0,
        "speed": 0,
        "reqCoordType": "WGS84GEO",
        "searchOption": "0",
        "resCoordType": "WGS84GEO",
        "sort": "index",
        "startX": a[0][1],
        "startY": a[0][2],
        "endX": a[1][1],
        "endY": a[1][2],
        "startName": a[0][0],
        "endName": a[0][0],
    }
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "appKey": "l7xxfdc75c1509a74ecdba02bf5e024ee9d5"
    }

    response = requests.post(url, json=payload, headers=headers).text

    api_data = json.loads(response).get('features')

    coor_raw_data = [api_data[i].get('geometry').get('coordinates') for i in range(0,len(api_data))]

    finish_data = []
    for data in coor_raw_data :
        # data = api_data[1].get('geometry').get('coordinates')
        if type(data[0]) == list :
            for i in data:
                finish_data.append(i)
        else :
            pass
    return pd.DataFrame(finish_data).T.values



In [ ]:
b = pd.DataFrame(finish_data).T.values

fig = go.Figure(
    go.Scattermapbox(
        lat=b[1],
        lon=b[0],
        mode="lines",
        marker=dict(symbol="circle", size=15, color='#black' , opacity=0.5),
    )
)
# fig.add_trace(fig_1.data[0])

fig.update_layout(
    mapbox=dict(
        accesstoken="pk.eyJ1IjoieWFuZ29vcyIsImEiOiJjbDNqd2tkN2IwbGdmM2pvNzF0c2M4NnZkIn0.J3IjPYg3w28cGiWkUD7bnA",
        # style='mapbox://styles/yangoos/cl3jubvl7000c14llgtoev0nm'
    ),
    margin=dict(l=20, r=20, t=20, b=20)
)
fig.show()


> ### 자전거 대여소 간 계산
* 해당 경우 검색 시 
* 지하철 -> 대여소 -> 대여소
* 대여소 -> 대여소 

In [ ]:


# 345ms
def st_to_st_new (starting_station, ending_station):

    lat_1=station.query('id==@starting_station')['위도'].values
    lon_1=station.query('id==@starting_station')['경도'].values
    lat_2=station.query('id==@ending_station')['위도'].values
    lon_2=station.query('id==@ending_station')['경도'].values
    dist_station = haversine_np(lat_1,lon_1,lat_2,lon_2)

    BM_1 = (seoul_bike['st_id1'] == starting_station) & (seoul_bike['st_id2'] == ending_station)
    BM_2 = (seoul_bike['st_id2'] == starting_station) & (seoul_bike['st_id1'] == ending_station)

    # 평균거리 계산
    sort_station = seoul_bike[BM_1 | BM_2]
    mean_station = sort_station[sort_station['dist'] < dist_station[0]*2]['riding_time'].value_counts()[:5].index.values.mean()
    num_station = len(sort_station)

    print(mean_station,num_station,f'{dist_station[0]/1000 :.2f}km' )

st_to_st_new(207, 754)


In [ ]:
## 옛날자료

def st_to_st(starting_station, ending_station):

    """
    starting_station = list or val이고 bike_station id가 들어감
    ending_station = list or val이고 bike_station id가 들어감
    """

    # int 또는 list로 starting station 받기
    if type(starting_station) == int:
        starting_station_list = list()
        starting_station_list.append(starting_station)
    else:
        starting_station_list = starting_station

    for starting_station in starting_station_list:
        BM_1 = (seoul_bike_sorting["st_id1"] == starting_station) & (
            seoul_bike_sorting["st_id2"] == ending_station
        )
        BM_2 = (seoul_bike_sorting["st_id2"] == starting_station) & (
            seoul_bike_sorting["st_id1"] == ending_station
        )

        # 평균거리 계산
        mean_station = (
            seoul_bike_sorting[BM_1 | BM_2]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)[:5]
            .index.values.mean()
        )

        # 이용기록 계산
        num_station = len(seoul_bike_sorting[BM_1 | BM_2])

        # 직선거리 계산
        lat_1 = station.query("id==@starting_station")["위도"].values
        lon_1 = station.query("id==@starting_station")["경도"].values
        lat_2 = station.query("id==@ending_station")["위도"].values
        lon_2 = station.query("id==@ending_station")["경도"].values
        dist_station = haversine_np(lat_1, lon_1, lat_2, lon_2)
        print(mean_station, num_station, f"{dist_station[0]/1000 :.2f}km")
    # return


st_to_st(counts_station, 754)
